In [ ]:
# Step 1: Import Libraries
import numpy as np
from scipy.linalg import svd
from scipy.io import loadmat
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Step 2: Load and Preprocess Velocity Data
from google.colab import drive
drive.mount('/content/drive')

# Load velocity data
velocity_data_U = loadmat('/content/drive/MyDrive/u.mat')['u']  # Replace with the correct path
velocity_data_V = loadmat('/content/drive/MyDrive/v.mat')['v']  # Replace with the correct path

# Reshape data
velocity_data_U = velocity_data_U.reshape(-1, velocity_data_U.shape[-1])  # n x d
velocity_data_V = velocity_data_V.reshape(-1, velocity_data_V.shape[-1])  # n x d

# Stack U and V as a data matrix (D)
D = np.concatenate([velocity_data_U, velocity_data_V], axis=1)
print(f"Shape of D: {D.shape}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Shape of D: (730800, 342)


In [ ]:
# Step 3: EOF Analysis Using SVD
U_svd, Sigma, Vt_svd = svd(D, full_matrices=False)

# Dimensionality reduction
k = 5  # Number of components to retain
U_k = U_svd[:, :k]
Sigma_k = np.diag(Sigma[:k])
Vt_k = Vt_svd[:k, :]

# Reconstruct the data with reduced components
D_reconstructed = np.dot(U_k, np.dot(Sigma_k, Vt_k))

In [ ]:
# Extract PCs for forecasting
PCs = Vt_svd.T
scaler = StandardScaler()
PCs_normalized = scaler.fit_transform(PCs)

# Step 4: Prepare Data for Forecasting
seq_length = 5  # Use past 5 steps for forecasting. Adjust as per needs.

def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
    return np.array(sequences)

X_PCs = create_sequences(PCs_normalized, seq_length)
y_PCs = PCs_normalized[seq_length:]

# Convert to PyTorch tensors
X_PCs_tensor = torch.tensor(X_PCs, dtype=torch.float32).permute(1, 0, 2)
y_PCs_tensor = torch.tensor(y_PCs, dtype=torch.float32)


In [ ]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, seq_len):
        super(TransformerModel, self).__init__()
        self.seq_len = seq_len
        self.input_dim = input_dim
        self.transformer = nn.Transformer(
            d_model=input_dim,
            nhead=1,
            num_encoder_layers=2,
            num_decoder_layers=2,
            batch_first=True  # Ensure batch-first processing
        )
        self.fc_out = nn.Linear(input_dim, input_dim)

    def forward(self, src, tgt):
        x = self.transformer(src, tgt)
        x = self.fc_out(x)
        return x

input_dim = PCs_normalized.shape[1]
model = TransformerModel(input_dim=input_dim, seq_len=seq_length)
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Step 6: Train the Transformer Model (Using EOF PCs)
epochs = 2  # Adjust as per needs
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_PCs_tensor, X_PCs_tensor)
    print(f"Output shape: {output.shape}")
    print(f"Target shape: {y_PCs_tensor.shape}")
    loss = criterion(output, y_PCs_tensor)
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(


Output shape: torch.Size([5, 337, 342])
Target shape: torch.Size([337, 342])


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([337, 342])) that is different to the input size (torch.Size([5, 337, 342])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Loss: 1.3376600742340088
Output shape: torch.Size([5, 337, 342])
Target shape: torch.Size([337, 342])
Epoch 2, Loss: 1.4202789068222046


In [ ]:
# Step 7: Train on Original Data (Without EOF)

# Downsample the data
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader, TensorDataset
step = 10  # Keep every 10th row
D_downsampled = D[::step]

# Reduce dimensionality with PCA
num_components = 50
pca = PCA(n_components=num_components)
D_reduced = pca.fit_transform(D_downsampled)

# Normalize in chunks
chunk_size = 10000
normalized_chunks = [
    StandardScaler().fit_transform(D_reduced[i:i + chunk_size])
    for i in range(0, D_reduced.shape[0], chunk_size)
]
D_normalized = np.vstack(normalized_chunks)

# Create sequences
seq_length = 5  # Adjusted for memory efficiency
X_original = create_sequences(D_normalized, seq_length)
y_original = D_normalized[seq_length:]

y_original_tensor = torch.tensor(y_original, dtype=torch.float32)
y_original_tensor = y_original_tensor[:len(X_original)]
X_original_tensor = torch.tensor(X_original, dtype=torch.float32)

# Create DataLoader for mini-batch processing
batch_size = 512  # Adjust based on memory capacity
dataset = TensorDataset(
    X_original_tensor,
    y_original_tensor
)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model_original = TransformerModel(input_dim=D_normalized.shape[1], seq_len=seq_length)
optimizer_original = Adam(model_original.parameters(), lr=0.001)
criterion = nn.MSELoss()

for epoch in range(epochs):
    epoch_loss = 0.0
    for X_batch, y_batch in dataloader:
        model_original.train()
        optimizer_original.zero_grad()
        output = model_original(X_batch, X_batch)
        output_last_step = output[:, -1, :]  # Use only the last time step
        loss = criterion(output_last_step, y_batch)
        loss.backward()
        optimizer_original.step()
        epoch_loss += loss.item()

    print(f'[Original Data] Epoch {epoch + 1}, Average Loss: {epoch_loss / len(dataloader)}')


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(


[Original Data] Epoch 1, Average Loss: 0.8262939303071348
[Original Data] Epoch 2, Average Loss: 0.6792456053353689


In [ ]:
# Step 8: Evaluate Performance
y_pred_PCs = model(X_PCs_tensor, X_PCs_tensor).detach().numpy()

# Select only the last prediction from the sequence for each sample
y_pred_PCs = y_pred_PCs[-1, :, :]  # Shape: (337, 342)

# Now, calculate the MSE and correlation
mse_PCs = mean_squared_error(y_PCs_tensor, y_pred_PCs)
correlation_PCs = np.corrcoef(y_PCs_tensor.flatten(), y_pred_PCs.flatten())[0, 1]

print(f'[EOF PCs] MSE: {mse_PCs}, Correlation: {correlation_PCs}')

# Evaluate original data model performance
y_pred_original = model_original(X_original_tensor, X_original_tensor).detach().numpy()
mse_original = mean_squared_error(y_original_tensor.squeeze().numpy(), y_pred_original.squeeze())
correlation_original = np.corrcoef(y_original_tensor.squeeze().numpy().flatten(), y_pred_original.flatten())[0, 1]

print(f'[Original Data] MSE: {mse_original}, Correlation: {correlation_original}')


[EOF PCs] MSE: 1.1274749040603638, Correlation: 0.0029440311177484236


In [ ]:
def diffusion_reconstruction(data, missing_rate=0.3):
    mask = np.random.rand(*data.shape) > missing_rate
    data_missing = data * mask

    for _ in range(10):
        data_missing = np.nan_to_num(data_missing)
        data_missing = np.roll(data_missing, 1, axis=0)
    return data_missing

D_reconstructed_diffusion = diffusion_reconstruction(D)
mse_diffusion = mean_squared_error(D, D_reconstructed_diffusion)

print(f'Diffusion Reconstruction MSE: {mse_diffusion}')

Diffusion Reconstruction MSE: 0.08645015954971313
